# Scrape Linkedin Data

In [1]:
# Make sure we have installed the dependency
! pip freeze | grep linkedin

linkedin-scraper==2.11.2


In [2]:
! google-chrome-stable --version

Google Chrome 114.0.5735.90 


In [3]:
from linkedin_scraper import JobSearch, Job, actions
from typing import List
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

import os
from pprint import pprint
import urllib
from time import sleep

def set_chrome_options() -> Options:
    """Sets chrome options for Selenium.
    Chrome options for headless browser is enabled.
    """
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_prefs = {}
    chrome_options.experimental_options["prefs"] = chrome_prefs
    chrome_prefs["profile.default_content_settings"] = {"images": 2}
    return chrome_options

class _JobSearch(JobSearch):
    def __init__(self, final_url=None, **kwargs):
        self.final_url = final_url
        self.current_url = None
        super().__init__(**kwargs)
    
    def search(self, search_term: str, page_n) -> List[Job]:
        if self.final_url is None:
            self.current_url = os.path.join(self.base_url, "search") + f"?keywords={urllib.parse.quote(search_term)}&refresh=true"
            self.driver.get(self.current_url)

            # Get redirection URL
            self.final_url = self.driver.current_url
        else:
            self.current_url = os.path.join(self.final_url, f"&start={25*(page_n-1)}")
            self.driver.get(self.current_url)
        
        self.scroll_to_bottom()
        self.focus()
        sleep(self.WAIT_FOR_ELEMENT_TIMEOUT)

        job_listing_class_name = "jobs-search-results-list"
        job_listing = self.wait_for_element_to_load(name=job_listing_class_name)

        self.scroll_class_name_element_to_page_percent(job_listing_class_name, 0.3)
        self.focus()
        sleep(self.WAIT_FOR_ELEMENT_TIMEOUT)

        self.scroll_class_name_element_to_page_percent(job_listing_class_name, 0.6)
        self.focus()
        sleep(self.WAIT_FOR_ELEMENT_TIMEOUT)

        self.scroll_class_name_element_to_page_percent(job_listing_class_name, 1)
        self.focus()
        sleep(self.WAIT_FOR_ELEMENT_TIMEOUT)

        job_results = []
        for job_card in self.wait_for_all_elements_to_load(name="job-card-list", base=job_listing):
            job = self.scrape_job_card(job_card)
            job_results.append(job)
        return job_results

def are_same(job1: Job, job2: Job):
    if job1.job_title == job2.job_title and job1.company == job2.company:
        return True
    return False

## 1. Scrape Job Search

Scrape the first 50 pages of the search result.

In [4]:
# Set up the lower-level services for scraping
driver = webdriver.Chrome(options=set_chrome_options())
actions.login(driver, os.environ["EMAIL"], os.environ["PWORD"]) # if email and password isnt given, it'll prompt in terminal
print("... Logged in.")
job_search = _JobSearch(driver=driver, close_on_complete=False, scrape=False)

... Logged in.


In [5]:
%%time
from selenium.common.exceptions import TimeoutException

N_PAGES = 50
SEARCH_KEYWORD = "data"

jobs = []
for page_n in range(1, N_PAGES+1):
    pprint(f"Searching jobs... Keyword: {SEARCH_KEYWORD}; Page {page_n}/{N_PAGES}")
    try:
        new_batch = job_search.search(SEARCH_KEYWORD, page_n)
    except TimeoutException:
        pprint(f"SKIPPED PAGE: {page_n}")
        continue

    # Check if the new batch of jobs are duplicates, 
    # which means we have gone through all the pages and should quit scraping.
    if jobs and are_same(new_batch[0], jobs[0]):
        pprint("Found duplicate results! All the pages have been scraped. Quiting...")
        break
        
    jobs.extend(new_batch)
    pprint(f"FINISHED PAGE: {page_n}")

'Searching jobs... Keyword: data; Page 1/50'
'FINISHED PAGE: 1'
'Searching jobs... Keyword: data; Page 2/50'
'FINISHED PAGE: 2'
'Searching jobs... Keyword: data; Page 3/50'
'FINISHED PAGE: 3'
'Searching jobs... Keyword: data; Page 4/50'
'FINISHED PAGE: 4'
'Searching jobs... Keyword: data; Page 5/50'
'FINISHED PAGE: 5'
'Searching jobs... Keyword: data; Page 6/50'
'FINISHED PAGE: 6'
'Searching jobs... Keyword: data; Page 7/50'
'FINISHED PAGE: 7'
'Searching jobs... Keyword: data; Page 8/50'
'FINISHED PAGE: 8'
'Searching jobs... Keyword: data; Page 9/50'
'FINISHED PAGE: 9'
'Searching jobs... Keyword: data; Page 10/50'
'FINISHED PAGE: 10'
'Searching jobs... Keyword: data; Page 11/50'
'FINISHED PAGE: 11'
'Searching jobs... Keyword: data; Page 12/50'
'FINISHED PAGE: 12'
'Searching jobs... Keyword: data; Page 13/50'
'FINISHED PAGE: 13'
'Searching jobs... Keyword: data; Page 14/50'
'FINISHED PAGE: 14'
'Searching jobs... Keyword: data; Page 15/50'
'FINISHED PAGE: 15'
'Searching jobs... Keyword: 

In [12]:
len(jobs)

379

In [14]:
# Save today's crawl temporarily
import pickle
import datetime

current_date = datetime.datetime.now().strftime("%Y-%m-%d")
fname = f"helsinki_data_jobs_{current_date}.pkl"
with open(f"../data/tmp/{fname}", "wb") as f:
    dicted_jobs = [job.to_dict() for job in jobs]
    pickle.dump(dicted_jobs,f)

## 2. Scrape job postings

In [8]:
import logging
from linkedin_scraper import Job, actions

from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class _Job(Job):
    def __init__(self, **kwargs):
       self.job_title = ""
       self.required_skills = ""
       self.job_type_1 = ""
       self.job_type_2 = ""
 
       super().__init__(**kwargs)
    
    def scrape_logged_in(self, close_on_complete=True):
        driver = self.driver
        
        driver.get(self.linkedin_url)
        self.focus()
        self.job_title = self.wait_for_element_to_load(by=By.XPATH, name="//*[contains(@class, 'jobs-unified-top-card__job-title')]").text.strip()
        self.company = self.wait_for_element_to_load(by=By.XPATH, name="//*[contains(@class, 'job-details-jobs-unified-top-card__primary-description')]//a[1]").text.strip()
        self.company_linkedin_url = self.wait_for_element_to_load(by=By.XPATH, name="//*[contains(@class, 'job-details-jobs-unified-top-card__primary-description')]//a").get_attribute("href")
        self.location = self.wait_for_element_to_load(by=By.XPATH, name="//*[contains(@class, 'job-details-jobs-unified-top-card__primary-description')]//*").text.strip()
        self.posted_date = self.wait_for_element_to_load(by=By.XPATH, name="//*[contains(@class, 'job-details-jobs-unified-top-card__primary-description')]//span[3]").text.strip()
        self.job_type_1 = self.wait_for_element_to_load(by=By.XPATH, name="//*[contains(@class, 'ui-label ui-label--accent-3 text-body-small')]/span").text.strip()
        self.job_description = self.wait_for_element_to_load(by=By.XPATH, name="//*[contains(@class, 'jobs-description')]").text.strip()
        
        try:
            self.required_skills = self.wait_for_element_to_load(by=By.XPATH, name="//*[contains(@class, 'job-details-how-you-match__skills-item')][1]//a").text.strip()
        except TimeoutException as e:
            logger.error(str(e))

        try:
            self.required_skills += self.wait_for_element_to_load(by=By.XPATH, name="//*[contains(@class, 'job-details-how-you-match__skills-item')][2]//a").text.strip()
        except TimeoutException as e:
            logger.error(str(e))

        try:
            self.job_type_2 = self.wait_for_element_to_load(by=By.XPATH, name="(//*[contains(@class, 'ui-label ui-label--accent-3 text-body-small')])[2]/span").text.strip()
        except TimeoutException:
            self.job_type_2 = ""
            
        try:
            self.applicant_count = self.wait_for_element_to_load(by=By.XPATH, name="jobs-unified-top-card__applicant-count").text.strip()
        except TimeoutException:
            self.applicant_count = 0
        
        try:
            self.benefits = self.wait_for_element_to_load(by=By.XPATH, name="//*[contains(@class, 'salary-main-rail-card')]").text.strip()
        except TimeoutException:
            self.benefits = ""

        if close_on_complete:
            driver.close()

In [9]:
from typing import List
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

import os
from pprint import pprint
import urllib
from time import sleep

def set_chrome_options() -> Options:
    """Sets chrome options for Selenium.
    Chrome options for headless browser is enabled.
    """
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_prefs = {}
    chrome_options.experimental_options["prefs"] = chrome_prefs
    chrome_prefs["profile.default_content_settings"] = {"images": 2}
    return chrome_options

In [10]:
# Set up low-level servies for scraping
driver = webdriver.Chrome(options=set_chrome_options())
actions.login(driver, os.environ["EMAIL"], os.environ["PWORD"]) 
print("... Logged in.")

... Logged in.


Ignore the error logs!

In [15]:
import pickle
import datetime

current_date = datetime.datetime.now().strftime("%Y-%m-%d")
fname = f"helsinki_data_jobs_{current_date}.pkl"

with open(f"../data/tmp/{fname}", "rb") as f:
    jobs = pickle.load(f)

print(len(jobs))

379


In [16]:
%%time
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException
from time import sleep

N_JOBS = len(jobs)

crawled_jobs = []
for i, job in enumerate(jobs):
    print(f"Crawling... Jobs {i+1}/{N_JOBS}")
    try:
        _crawled_job = _Job(linkedin_url=job.get("linkedin_url"), driver=driver, close_on_complete=False, scrape=True)
        crawled_jobs.append(_crawled_job)
        sleep(1)
    except StaleElementReferenceException or TimeoutException:
        print(f"... Skipped Job {i+1}/{N_JOBS}.")
        sleep(1)
        continue

Crawling... Jobs 1/379
Crawling... Jobs 2/379
Crawling... Jobs 3/379


ERROR:__main__:Message: 
Stacktrace:
#0 0x55d23404e4e3 <unknown>
#1 0x55d233d7dc76 <unknown>
#2 0x55d233db9c96 <unknown>
#3 0x55d233db9dc1 <unknown>
#4 0x55d233df37f4 <unknown>
#5 0x55d233dd903d <unknown>
#6 0x55d233df130e <unknown>
#7 0x55d233dd8de3 <unknown>
#8 0x55d233dae2dd <unknown>
#9 0x55d233daf34e <unknown>
#10 0x55d23400e3e4 <unknown>
#11 0x55d2340123d7 <unknown>
#12 0x55d23401cb20 <unknown>
#13 0x55d234013023 <unknown>
#14 0x55d233fe11aa <unknown>
#15 0x55d2340376b8 <unknown>
#16 0x55d234037847 <unknown>
#17 0x55d234047243 <unknown>
#18 0x7f4c3d494ac3 <unknown>



Crawling... Jobs 4/379
Crawling... Jobs 5/379
Crawling... Jobs 6/379
Crawling... Jobs 7/379
Crawling... Jobs 8/379
Crawling... Jobs 9/379
Crawling... Jobs 10/379
Crawling... Jobs 11/379
Crawling... Jobs 12/379
Crawling... Jobs 13/379


ERROR:__main__:Message: 
Stacktrace:
#0 0x55d23404e4e3 <unknown>
#1 0x55d233d7dc76 <unknown>
#2 0x55d233db9c96 <unknown>
#3 0x55d233db9dc1 <unknown>
#4 0x55d233df37f4 <unknown>
#5 0x55d233dd903d <unknown>
#6 0x55d233df130e <unknown>
#7 0x55d233dd8de3 <unknown>
#8 0x55d233dae2dd <unknown>
#9 0x55d233daf34e <unknown>
#10 0x55d23400e3e4 <unknown>
#11 0x55d2340123d7 <unknown>
#12 0x55d23401cb20 <unknown>
#13 0x55d234013023 <unknown>
#14 0x55d233fe11aa <unknown>
#15 0x55d2340376b8 <unknown>
#16 0x55d234037847 <unknown>
#17 0x55d234047243 <unknown>
#18 0x7f4c3d494ac3 <unknown>



Crawling... Jobs 14/379
Crawling... Jobs 15/379
Crawling... Jobs 16/379
Crawling... Jobs 17/379
Crawling... Jobs 18/379
Crawling... Jobs 19/379
Crawling... Jobs 20/379
Crawling... Jobs 21/379
Crawling... Jobs 22/379


...

ERROR:__main__:Message: 
Stacktrace:
#0 0x55d23404e4e3 <unknown>
#1 0x55d233d7dc76 <unknown>
#2 0x55d233db9c96 <unknown>
#3 0x55d233db9dc1 <unknown>
#4 0x55d233df37f4 <unknown>
#5 0x55d233dd903d <unknown>
#6 0x55d233df130e <unknown>
#7 0x55d233dd8de3 <unknown>
#8 0x55d233dae2dd <unknown>
#9 0x55d233daf34e <unknown>
#10 0x55d23400e3e4 <unknown>
#11 0x55d2340123d7 <unknown>
#12 0x55d23401cb20 <unknown>
#13 0x55d234013023 <unknown>
#14 0x55d233fe11aa <unknown>
#15 0x55d2340376b8 <unknown>
#16 0x55d234037847 <unknown>
#17 0x55d234047243 <unknown>
#18 0x7f4c3d494ac3 <unknown>

ERROR:__main__:Message: 
Stacktrace:
#0 0x55d23404e4e3 <unknown>
#1 0x55d233d7dc76 <unknown>
#2 0x55d233db9c96 <unknown>
#3 0x55d233db9dc1 <unknown>
#4 0x55d233df37f4 <unknown>
#5 0x55d233dd903d <unknown>
#6 0x55d233df130e <unknown>
#7 0x55d233dd8de3 <unknown>
#8 0x55d233dae2dd <unknown>
#9 0x55d233daf34e <unknown>
#10 0x55d23400e3e4 <unknown>
#11 0x55d2340123d7 <unknown>
#12 0x55d23401cb20 <unknown>
#13 0x55d23401302

Crawling... Jobs 49/379
Crawling... Jobs 50/379
Crawling... Jobs 51/379
Crawling... Jobs 52/379
Crawling... Jobs 53/379
Crawling... Jobs 54/379
Crawling... Jobs 55/379
Crawling... Jobs 56/379


ERROR:__main__:Message: 
Stacktrace:
#0 0x55d23404e4e3 <unknown>
#1 0x55d233d7dc76 <unknown>
#2 0x55d233db9c96 <unknown>
#3 0x55d233db9dc1 <unknown>
#4 0x55d233df37f4 <unknown>
#5 0x55d233dd903d <unknown>
#6 0x55d233df130e <unknown>
#7 0x55d233dd8de3 <unknown>
#8 0x55d233dae2dd <unknown>
#9 0x55d233daf34e <unknown>
#10 0x55d23400e3e4 <unknown>
#11 0x55d2340123d7 <unknown>
#12 0x55d23401cb20 <unknown>
#13 0x55d234013023 <unknown>
#14 0x55d233fe11aa <unknown>
#15 0x55d2340376b8 <unknown>
#16 0x55d234037847 <unknown>
#17 0x55d234047243 <unknown>
#18 0x7f4c3d494ac3 <unknown>

ERROR:__main__:Message: 
Stacktrace:
#0 0x55d23404e4e3 <unknown>
#1 0x55d233d7dc76 <unknown>
#2 0x55d233db9c96 <unknown>
#3 0x55d233db9dc1 <unknown>
#4 0x55d233df37f4 <unknown>
#5 0x55d233dd903d <unknown>
#6 0x55d233df130e <unknown>
#7 0x55d233dd8de3 <unknown>
#8 0x55d233dae2dd <unknown>
#9 0x55d233daf34e <unknown>
#10 0x55d23400e3e4 <unknown>
#11 0x55d2340123d7 <unknown>
#12 0x55d23401cb20 <unknown>
#13 0x55d23401302

Crawling... Jobs 57/379


WebDriverException: Message: unknown error: net::ERR_CONNECTION_REFUSED
  (Session info: headless chrome=114.0.5735.90)
Stacktrace:
#0 0x55d23404e4e3 <unknown>
#1 0x55d233d7dc76 <unknown>
#2 0x55d233d75c7f <unknown>
#3 0x55d233d67ca2 <unknown>
#4 0x55d233d69412 <unknown>
#5 0x55d233d680ca <unknown>
#6 0x55d233d67168 <unknown>
#7 0x55d233d66fa0 <unknown>
#8 0x55d233d659bf <unknown>
#9 0x55d233d65fed <unknown>
#10 0x55d233d7fb06 <unknown>
#11 0x55d233df19e5 <unknown>
#12 0x55d233dd9012 <unknown>
#13 0x55d233df130e <unknown>
#14 0x55d233dd8de3 <unknown>
#15 0x55d233dae2dd <unknown>
#16 0x55d233daf34e <unknown>
#17 0x55d23400e3e4 <unknown>
#18 0x55d2340123d7 <unknown>
#19 0x55d23401cb20 <unknown>
#20 0x55d234013023 <unknown>
#21 0x55d233fe11aa <unknown>
#22 0x55d2340376b8 <unknown>
#23 0x55d234037847 <unknown>
#24 0x55d234047243 <unknown>
#25 0x7f4c3d494ac3 <unknown>


In [17]:
import pandas as pd

In [18]:
df_crawled_jobs = pd.DataFrame([vars(job) for job in crawled_jobs]
                              ).drop(columns=["driver"]
                              ).drop_duplicates("linkedin_url")

In [19]:
df_crawled_jobs

,job_title,required_skills,job_type_1,job_type_2,linkedin_url,company,company_linkedin_url,location,posted_date,applicant_count,job_description,benefits
0,QARA Specialist / Senior Specialist,"EnglishAttention to Detail, Communication, IEC...",Hybrid,Full-time,https://www.linkedin.com/jobs/view/3745632980/...,Topcon Healthcare Europe,https://www.linkedin.com/company/topconhealthc...,"Topcon Healthcare Europe · Oulu, North Ostrobo...",4 days ago,0,About the job\nDo you want to have a direct im...,
1,"Juristi-trainee, Data ja tietosuoja","Analytical Skills, Big Data, Data Analysis, Da...",Hybrid,Full-time,https://www.linkedin.com/jobs/view/3741023935/...,OP Financial Group,https://www.linkedin.com/company/op-financial-...,"OP Financial Group · Helsinki, Uusimaa, Finlan...",2 weeks ago,0,About the job\nHaku OP Ryhmän vuoden 2024 Kiit...,
2,Helsinki Associate Program - Privacy (Cyber & ...,"Communication, Organization Skills, and Proble...",On-site,Full-time,https://www.linkedin.com/jobs/view/3748742188/...,Advisense,https://www.linkedin.com/company/advisense/life,"Advisense · Helsinki, Uusimaa, Finland Reposte...",Reposted 1 week ago,0,About the job\nWe are looking for a new collea...,
3,Azure Data Engineer,"Artificial Intelligence (AI), Cloud Computing,...",On-site,Full-time,https://www.linkedin.com/jobs/view/3684400225/...,Cloud1 Oy,https://www.linkedin.com/company/cloud1-oy/life,"Cloud1 Oy · Helsinki, Uusimaa, Finland Reposte...",Reposted 4 weeks ago,0,About the job\nCloud1 tunnetaan vaativien inte...,
4,"Data Scientist, Pohjola Insurance Analytics, H...","Artificial Intelligence (AI), Data Science, an...",Hybrid,Full-time,https://www.linkedin.com/jobs/view/3757316577/...,OP Financial Group,https://www.linkedin.com/company/op-financial-...,"OP Financial Group · Helsinki, Uusimaa, Finlan...",1 hour ago,0,About the job\nAre you skilled in handling ext...,
5,"Data Scientist, Pohjola Vakuutuksen analytiikk...","Data Analytics, Data Engineering, Data Science...",Hybrid,Full-time,https://www.linkedin.com/jobs/view/3757316580/...,OP Financial Group,https://www.linkedin.com/company/op-financial-...,"OP Financial Group · Helsinki, Uusimaa, Finlan...",1 hour ago,0,About the job\nOletko taitava käsittelemään la...,
6,Data Engineer (Level Up),"Data Warehousing, Finnish, and SQLData Visuali...",Hybrid,Full-time,https://www.linkedin.com/jobs/view/3744740320/...,Loihde Advance,https://www.linkedin.com/company/loihdeadvance...,"Loihde Advance · Uusimaa, Finland 2 weeks ago...",2 weeks ago,0,About the job\nOnko sinulle jo kertynyt jo väh...,
7,Quantexa Monitoring Specialist,"Analytical SkillsCommunication, Stakeholder Ma...",On-site,Full-time,https://www.linkedin.com/jobs/view/3731087824/...,Nordea,https://www.linkedin.com/company/nordea/life,"Nordea · Helsinki, Uusimaa, Finland Reposted ...",Reposted 2 days ago,0,About the job\nJob ID: 19733 \n Do you want to...,
8,Machine learning expert,"Artificial Intelligence (AI), Data Science, ML...",Hybrid,Full-time,https://www.linkedin.com/jobs/view/3751462279/...,Verso Vision,https://www.linkedin.com/company/verso-vision/...,"Verso Vision · Lappeenranta, South Karelia, Fi...",1 day ago,0,About the job\nWe are looking for a Machine Le...,
9,"Data Architect, Digital Society","Data Engineering, Extract, Transform, Load (ET...",Full-time,,https://www.linkedin.com/jobs/view/3755141358/...,CGI,https://www.linkedin.com/company/cgi/life,"CGI · Helsinki, Uusimaa, Finland 2 days ago ...",2 days ago,0,About the job\nPosition Description\n\nVuonna ...,


In [20]:
# df_crawled_jobs.to_csv(f"../data/crawled_jobs_1-{len(crawled_jobs}_checkpoint.csv", index=False)

### 2.1 Continue from the failed point

In [21]:
# In case session expiration
driver = webdriver.Chrome(options=set_chrome_options())
actions.login(driver, os.environ["EMAIL"], os.environ["PWORD"]) 
print("... Logged in.")

... Logged in.


In [34]:
%%time
# Continue
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException, WebDriverException

CONTINUE_FROM = 78

for i, job in enumerate(jobs):
    if i+1<CONTINUE_FROM:
        continue
        
    print(f"Crawling... Jobs {i+1}/{N_JOBS}")
    try:
        _crawled_job = _Job(linkedin_url=job.get("linkedin_url"), driver=driver, close_on_complete=False, scrape=True)
        crawled_jobs.append(_crawled_job)
        sleep(1)
    except (StaleElementReferenceException, TimeoutException):
        print(f"... Skipped Job {i+1}/{N_JOBS}.")
        sleep(1)
        continue
    except (WebDriverException, TypeError):
        print("Session expired. Logging in...") 
        driver = webdriver.Chrome(options=set_chrome_options())
        actions.login(driver, os.environ["EMAIL"], os.environ["PWORD"]) 
        print("... Logged in.")
        
        try: #Redo crawling
            _crawled_job = _Job(linkedin_url=job.get("linkedin_url"), driver=driver, close_on_complete=False, scrape=True)
            crawled_jobs.append(_crawled_job)
        except:
            print(f"... Skipped Job {i+1}/{N_JOBS}.")
            continue

Crawling... Jobs 98/379
Crawling... Jobs 99/379
Crawling... Jobs 100/379


ERROR:__main__:Message: 
Stacktrace:
#0 0x5604d51fe4e3 <unknown>
#1 0x5604d4f2dc76 <unknown>
#2 0x5604d4f69c96 <unknown>
#3 0x5604d4f69dc1 <unknown>
#4 0x5604d4fa37f4 <unknown>
#5 0x5604d4f8903d <unknown>
#6 0x5604d4fa130e <unknown>
#7 0x5604d4f88de3 <unknown>
#8 0x5604d4f5e2dd <unknown>
#9 0x5604d4f5f34e <unknown>
#10 0x5604d51be3e4 <unknown>
#11 0x5604d51c23d7 <unknown>
#12 0x5604d51ccb20 <unknown>
#13 0x5604d51c3023 <unknown>
#14 0x5604d51911aa <unknown>
#15 0x5604d51e76b8 <unknown>
#16 0x5604d51e7847 <unknown>
#17 0x5604d51f7243 <unknown>
#18 0x7f31a8e94ac3 <unknown>



Crawling... Jobs 101/379
Crawling... Jobs 102/379
Crawling... Jobs 103/379
Crawling... Jobs 104/379
Crawling... Jobs 105/379
Crawling... Jobs 106/379
Crawling... Jobs 107/379
Crawling... Jobs 108/379
Crawling... Jobs 109/379
Crawling... Jobs 110/379
Crawling... Jobs 111/379
Crawling... Jobs 112/379
Crawling... Jobs 113/379


ERROR:__main__:Message: 
Stacktrace:
#0 0x5604d51fe4e3 <unknown>
#1 0x5604d4f2dc76 <unknown>
#2 0x5604d4f69c96 <unknown>
#3 0x5604d4f69dc1 <unknown>
#4 0x5604d4fa37f4 <unknown>
#5 0x5604d4f8903d <unknown>
#6 0x5604d4fa130e <unknown>
#7 0x5604d4f88de3 <unknown>
#8 0x5604d4f5e2dd <unknown>
#9 0x5604d4f5f34e <unknown>
#10 0x5604d51be3e4 <unknown>
#11 0x5604d51c23d7 <unknown>
#12 0x5604d51ccb20 <unknown>
#13 0x5604d51c3023 <unknown>
#14 0x5604d51911aa <unknown>
#15 0x5604d51e76b8 <unknown>
#16 0x5604d51e7847 <unknown>
#17 0x5604d51f7243 <unknown>
#18 0x7f31a8e94ac3 <unknown>

ERROR:__main__:Message: 
Stacktrace:
#0 0x5604d51fe4e3 <unknown>
#1 0x5604d4f2dc76 <unknown>
#2 0x5604d4f69c96 <unknown>
#3 0x5604d4f69dc1 <unknown>
#4 0x5604d4fa37f4 <unknown>
#5 0x5604d4f8903d <unknown>
#6 0x5604d4fa130e <unknown>
#7 0x5604d4f88de3 <unknown>
#8 0x5604d4f5e2dd <unknown>
#9 0x5604d4f5f34e <unknown>
#10 0x5604d51be3e4 <unknown>
#11 0x5604d51c23d7 <unknown>
#12 0x5604d51ccb20 <unknown>
#13 0x5604d51c302

Crawling... Jobs 114/379
Crawling... Jobs 115/379
Crawling... Jobs 116/379


...

Crawling... Jobs 295/379
Crawling... Jobs 296/379


ERROR:__main__:Message: 
Stacktrace:
#0 0x5604d51fe4e3 <unknown>
#1 0x5604d4f2dc76 <unknown>
#2 0x5604d4f69c96 <unknown>
#3 0x5604d4f69dc1 <unknown>
#4 0x5604d4fa37f4 <unknown>
#5 0x5604d4f8903d <unknown>
#6 0x5604d4fa130e <unknown>
#7 0x5604d4f88de3 <unknown>
#8 0x5604d4f5e2dd <unknown>
#9 0x5604d4f5f34e <unknown>
#10 0x5604d51be3e4 <unknown>
#11 0x5604d51c23d7 <unknown>
#12 0x5604d51ccb20 <unknown>
#13 0x5604d51c3023 <unknown>
#14 0x5604d51911aa <unknown>
#15 0x5604d51e76b8 <unknown>
#16 0x5604d51e7847 <unknown>
#17 0x5604d51f7243 <unknown>
#18 0x7f31a8e94ac3 <unknown>



Crawling... Jobs 297/379
Crawling... Jobs 298/379
Crawling... Jobs 299/379
Crawling... Jobs 300/379
Crawling... Jobs 301/379
Crawling... Jobs 302/379
Crawling... Jobs 303/379
Crawling... Jobs 304/379
Crawling... Jobs 305/379
Crawling... Jobs 306/379
Crawling... Jobs 307/379


ERROR:__main__:Message: 
Stacktrace:
#0 0x5604d51fe4e3 <unknown>
#1 0x5604d4f2dc76 <unknown>
#2 0x5604d4f69c96 <unknown>
#3 0x5604d4f69dc1 <unknown>
#4 0x5604d4fa37f4 <unknown>
#5 0x5604d4f8903d <unknown>
#6 0x5604d4fa130e <unknown>
#7 0x5604d4f88de3 <unknown>
#8 0x5604d4f5e2dd <unknown>
#9 0x5604d4f5f34e <unknown>
#10 0x5604d51be3e4 <unknown>
#11 0x5604d51c23d7 <unknown>
#12 0x5604d51ccb20 <unknown>
#13 0x5604d51c3023 <unknown>
#14 0x5604d51911aa <unknown>
#15 0x5604d51e76b8 <unknown>
#16 0x5604d51e7847 <unknown>
#17 0x5604d51f7243 <unknown>
#18 0x7f31a8e94ac3 <unknown>



Crawling... Jobs 308/379
Crawling... Jobs 309/379
Crawling... Jobs 310/379
Crawling... Jobs 311/379
Crawling... Jobs 312/379
Crawling... Jobs 313/379
Crawling... Jobs 314/379
Crawling... Jobs 315/379
Crawling... Jobs 316/379
Crawling... Jobs 317/379
Crawling... Jobs 318/379
Crawling... Jobs 319/379
Crawling... Jobs 320/379
Crawling... Jobs 321/379
Crawling... Jobs 322/379
Crawling... Jobs 323/379
Crawling... Jobs 324/379
Crawling... Jobs 325/379
Crawling... Jobs 326/379
Crawling... Jobs 327/379
Crawling... Jobs 328/379
Crawling... Jobs 329/379
Crawling... Jobs 330/379
Crawling... Jobs 331/379
Crawling... Jobs 332/379
Crawling... Jobs 333/379
Crawling... Jobs 334/379
Crawling... Jobs 335/379
Crawling... Jobs 336/379
Crawling... Jobs 337/379
Crawling... Jobs 338/379
Crawling... Jobs 339/379
Crawling... Jobs 340/379
Crawling... Jobs 341/379
Crawling... Jobs 342/379
Crawling... Jobs 343/379
Crawling... Jobs 344/379
Crawling... Jobs 345/379
Crawling... Jobs 346/379
Crawling... Jobs 347/379


In [35]:
import pandas as pd
df_crawled_jobs = pd.DataFrame([vars(job) for job in crawled_jobs]).drop(columns=["driver"]).drop_duplicates("linkedin_url")
df_crawled_jobs

,job_title,required_skills,job_type_1,job_type_2,linkedin_url,company,company_linkedin_url,location,posted_date,applicant_count,job_description,benefits
0,QARA Specialist / Senior Specialist,"EnglishAttention to Detail, Communication, IEC...",Hybrid,Full-time,https://www.linkedin.com/jobs/view/3745632980/...,Topcon Healthcare Europe,https://www.linkedin.com/company/topconhealthc...,"Topcon Healthcare Europe · Oulu, North Ostrobo...",4 days ago,0,About the job\nDo you want to have a direct im...,
1,"Juristi-trainee, Data ja tietosuoja","Analytical Skills, Big Data, Data Analysis, Da...",Hybrid,Full-time,https://www.linkedin.com/jobs/view/3741023935/...,OP Financial Group,https://www.linkedin.com/company/op-financial-...,"OP Financial Group · Helsinki, Uusimaa, Finlan...",2 weeks ago,0,About the job\nHaku OP Ryhmän vuoden 2024 Kiit...,
2,Helsinki Associate Program - Privacy (Cyber & ...,"Communication, Organization Skills, and Proble...",On-site,Full-time,https://www.linkedin.com/jobs/view/3748742188/...,Advisense,https://www.linkedin.com/company/advisense/life,"Advisense · Helsinki, Uusimaa, Finland Reposte...",Reposted 1 week ago,0,About the job\nWe are looking for a new collea...,
3,Azure Data Engineer,"Artificial Intelligence (AI), Cloud Computing,...",On-site,Full-time,https://www.linkedin.com/jobs/view/3684400225/...,Cloud1 Oy,https://www.linkedin.com/company/cloud1-oy/life,"Cloud1 Oy · Helsinki, Uusimaa, Finland Reposte...",Reposted 4 weeks ago,0,About the job\nCloud1 tunnetaan vaativien inte...,
4,"Data Scientist, Pohjola Insurance Analytics, H...","Artificial Intelligence (AI), Data Science, an...",Hybrid,Full-time,https://www.linkedin.com/jobs/view/3757316577/...,OP Financial Group,https://www.linkedin.com/company/op-financial-...,"OP Financial Group · Helsinki, Uusimaa, Finlan...",1 hour ago,0,About the job\nAre you skilled in handling ext...,
...,...,...,...,...,...,...,...,...,...,...,...,...
370,Cloud Engineer,Microsoft Azure and Python (Programming Langua...,Hybrid,Full-time,https://www.linkedin.com/jobs/view/3742666783/...,Mitsubishi Logisnext Europe Oy,https://www.linkedin.com/company/mletyopaikat/...,"Mitsubishi Logisnext Europe Oy · Järvenpää, Uu...",1 week ago,0,About the job\nAre you a DevSecOps developer w...,
371,Senior Machine Learning Engineer,"Artificial Intelligence (AI), Deep Learning, M...",Hybrid,Full-time,https://www.linkedin.com/jobs/view/3754184529/...,bubb,https://www.linkedin.com/company/bubb/life,bubb · Helsinki Metropolitan Area 4 days ago ...,4 days ago,0,About the job\nBubb are partnered with an amaz...,
372,AI Engineer Academy - Finland,"Artificial Intelligence (AI), Computer Science...",Full-time,,https://www.linkedin.com/jobs/view/3746215896/...,Avanade,https://www.linkedin.com/company/avanade/life,"Avanade · Helsinki, Uusimaa, Finland 1 week a...",1 week ago,0,About the job\nJob Description\n\nAre you inte...,
373,"Data Engineer, OP Life Insurance","Data Analytics, Data Engineering, Data Warehou...",Hybrid,Full-time,https://www.linkedin.com/jobs/view/3749553229/...,OP Financial Group,https://www.linkedin.com/company/op-financial-...,"OP Financial Group · Helsinki, Uusimaa, Finlan...",1 week ago,0,About the job\nAre you ready to embark on an e...,


In [36]:
# Save today's crawl
import datetime

current_date = datetime.datetime.now().strftime("%Y-%m-%d")
fname = f"../data/crawled_jobs_{current_date}.csv"

df_crawled_jobs.to_csv(fname, index=False)